In [1]:
import sys
sys.path.append('/oak/stanford/groups/rbaltman/luyang/scripts/POPDx/code/')
%load_ext autoreload
%autoreload 2

### ICD-10 embeddings

In [2]:
from embeddings import *

In [3]:
import transformers
print(transformers.__version__)

3.2.0


In [4]:
l2i, i2l, onto_embeddings = ontology_emb(dim=500, ICD_network_file = '../data/19.csv', save_dir = './embeddings/', use_pretrain = True)

In [6]:
biboert_embeddings, biboert_embeddings_dict = run_bert(use_pretrain = False)

cuda:0
Batch # 1
Batch # 2
Batch # 3
Batch # 4
Batch # 5
Batch # 6
Batch # 7
Batch # 8
Batch # 9
Batch # 10
Batch # 11
Batch # 12
Batch # 13
Batch # 14
Batch # 15
Batch # 16
Batch # 17
Batch # 18
Batch # 19
Batch # 20
Batch # 21
Batch # 22
Batch # 23
Batch # 24
Batch # 25
Batch # 26
Batch # 27
Batch # 28
Batch # 29
Batch # 30
Batch # 31
Batch # 32
Batch # 33
Batch # 34
Batch # 35
Batch # 36
Batch # 37
Batch # 38
Batch # 39
Batch # 40
Batch # 41
Batch # 42
Batch # 43
Batch # 44
Batch # 45
Batch # 46
Batch # 47
Batch # 48
Batch # 49
Batch # 50
Batch # 51
Batch # 52
Batch # 53
Batch # 54
Batch # 55
Batch # 56
Batch # 57
Batch # 58
Batch # 59
Batch # 60
Batch # 61
Batch # 62
Batch # 63
Batch # 64
Batch # 65
Batch # 66
Batch # 67
Batch # 68
Batch # 69
Batch # 70
Batch # 71
Batch # 72
Batch # 73
Batch # 74
Batch # 75
Batch # 76
Batch # 77
Batch # 78
Batch # 79
Batch # 80
Batch # 81
Batch # 82
Batch # 83
Batch # 84
Batch # 85
Batch # 86
Batch # 87
Batch # 88
Batch # 89
Batch # 90
Batch # 91
B

In [7]:
with open('../data/mc_icd10_labels.txt','r') as f:
    labels = f.readlines()
labels = [x.strip() for x in labels] 
labels_idx = [l2i[l] for l in labels]
onto_embeddings = onto_embeddings[labels_idx, :]
print(onto_embeddings.shape,  biboert_embeddings.shape)

(12803, 500) (12803, 768)


In [8]:
Y_emb_concat = np.concatenate((onto_embeddings, biboert_embeddings), axis=1)
print(Y_emb_concat.shape)

(12803, 1268)


In [9]:
np.save('../data/icd10_label_embed.npy', Y_emb_concat)

### Phecode embeddings


In [11]:
df_phecode = pd.read_csv('../data/Phecode_map_icd10.csv')

In [12]:
df_phecode.head()

,ICD10,PHECODE,Exl. Phecodes,Excl. Phenotypes
0,A009,8.0,001-009.99,Intestinal infection
1,A010,8.5,001-009.99,Intestinal infection
2,A011,8.0,001-009.99,Intestinal infection
3,A012,8.0,001-009.99,Intestinal infection
4,A013,8.0,001-009.99,Intestinal infection


In [18]:
with open('../data/phecode_labels.txt', 'r') as f:
    phecode_labels = f.readlines() 
phecode_labels = [float(x.strip()) for x in phecode_labels] 

In [22]:
phecode_labels_emb = np.zeros((1538, 1268))
for i,phelabel in enumerate(phecode_labels):
    #print(phelabel)
    corr_icd10 = df_phecode.loc[df_phecode.PHECODE==phelabel].ICD10.tolist()
    corr_idx = []
    for l in corr_icd10:
        corr_idx.append(labels.index(l))
    assert (corr_icd10 == np.array(labels)[corr_idx]).all()
    phecode_labels_emb[i,:] = Y_emb_concat[corr_idx, :].mean(axis=0)

In [21]:
np.save('../data/phecode_label_embed.npy', phecode_labels_emb)